# Worksheet 07

Name:  Di Wang
UID:  U22721196

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[4.4163964898046775, 4.6926257063803885, 7.155366500453685, 4.306435542425865, 4.864392579086808, 5.508065682336184, 4.441990042570537, 3.643448042686269, 5.996627888989444, 5.400024767715278]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
mean2 = 8
var2 = 1
c2 = np.random.normal(mean2, np.sqrt(var2), 10).tolist()
print(c2)

[8.02670851202366, 6.996033228418267, 8.658297070289873, 6.9743627550228435, 7.28476101579438, 8.928034731812248, 9.807724130184065, 9.59871973483993, 6.267793863812563, 8.886503156365356]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[8.886503156365356, 5.400024767715278, 6.267793863812563, 5.996627888989444, 9.59871973483993, 9.807724130184065, 8.928034731812248, 3.643448042686269, 7.28476101579438, 6.9743627550228435]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Number of components (K): 2

Means of each component (μ): 5, 8

Covariance matrices of each component (Σ): Unknown

Mixture weights (π): Unknown

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [len(c1) / (len(c1) + len(c2)), len(c2) / (len(c1) + len(c2))]
mean = [sum(c1) / len(c1), sum(c2) / len(c2)]
var = [sum(map(lambda x: (x - mean[0])**2, c1)) / len(c1),
       sum(map(lambda x: (x - mean[1])**2, c2)) / len(c2)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.400024767715278, 6.267793863812563, 5.996627888989444, 3.643448042686269, 7.28476101579438, 6.9743627550228435]
[8.886503156365356, 9.59871973483993, 9.807724130184065, 8.928034731812248]
P(C_1) = 0.6,  P(C_2) = 0.4
mean_1 = 5.927836389003463,  mean_2 = 9.3052454383004
var_1 = 1.4256300761784946,  var_2 = 0.16406125357398743


Comparing with true values, they are fairly close to the true values.

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  8.886503156365356
probability of observing that point if it came from cluster 0 =  0.032481975320072944
probability of observing that point if it came from cluster 1 =  0.09360450247295346
point =  5.400024767715278
probability of observing that point if it came from cluster 0 =  0.26129958629340094
probability of observing that point if it came from cluster 1 =  2.236446338766679e-123
point =  6.267793863812563
probability of observing that point if it came from cluster 0 =  0.27199154912360796
probability of observing that point if it came from cluster 1 =  8.98837169946903e-75
point =  5.996627888989444
probability of observing that point if it came from cluster 0 =  0.27951016926174205
probability of observing that point if it came from cluster 1 =  1.1768337509341815e-88
point =  9.59871973483993
probability of observing that point if it came from cluster 0 =  0.010166484309529522
probability of observing that point if it came from cluster 1 =  0.4909846228477396
point = 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [sum([x * (y - mean[0])**2 for x, y in zip(prob_c0_x, data)]) / sum(prob_c0_x),
       sum([x * (y - mean[1])**2 for x, y in zip(prob_c1_x, data)]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.6898486470780194,  P(C_2) = 0.31015135292198065
mean_1 = 6.359824842626049,  mean_2 = 9.322814226481356
var_1 = 2.5106440008729787,  var_2 = 0.15571083866718793


Comparing these values from the original ones you got from K means, we can see that they should be closer to the true values. Because the EM algorithm is attempting to iteratively improve the estimates based on the available data.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [29]:
import math

def norm_pdf(x, mean, variance):
    coeff = 1.0 / (math.sqrt(2.0 * math.pi) * math.sqrt(variance))
    exponent = math.exp(-(math.pow(x - mean, 2) / (2 * variance)))
    return coeff * exponent

prob_c0_x = [norm_pdf(x, mean[0], var[0]) * prob_c[0] for x in data]
prob_c1_x = [norm_pdf(x, mean[1], var[1]) * prob_c[1] for x in data]
sum_prob_cx = [prob_c0_x[i] + prob_c1_x[i] for i in range(len(data))]
prob_c_x = [[prob_c0_x[i]/sum_prob_cx[i], prob_c1_x[i]/sum_prob_cx[i]] for i in range(len(data))]

print(prob_c_x)

[[0.22255168306775527, 0.7774483169322448], [1.0, 7.523838049712674e-22], [0.9999999999998255, 1.7446191264649615e-13], [0.9999999999999993, 6.906968337176284e-16], [0.08050625047774165, 0.9194937495222584], [0.09946248205125019, 0.9005375179487498], [0.19720851432559586, 0.8027914856744042], [1.0, 8.186706559734885e-45], [0.9999965482555135, 3.4517444865208653e-06], [0.9999999603780139, 3.962198612174554e-08]]


Based on the previous results, the updated P(C_j | X_i) should be similar to the previous values. However, depending on the new data and the convergence of the algorithm, the values may shift slightly.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [31]:
# Create a list to hold the labels
labels = []

# For each data point
for i in range(len(data)):
    # Compute the P(C_0 | X_i) and P(C_1 | X_i) values
    prob_c0_xi = prob_c_x[i][0]
    prob_c1_xi = prob_c_x[i][1]
    
    # Assign the label based on the larger probability
    if prob_c0_xi >= prob_c1_xi:
        labels.append(0)
    else:
        labels.append(1)


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      N       |
| A  C |      Y       |
| A  D |      N       |
| A  E |      N       |
| B  C |      Y       |
| B  D |      Y       |
| B  E |      Y       |
| C  D |      Y       |
| C  E |      Y       |
| D  E |      N       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

N(N-1)/2

This formula accounts for the fact that for every point in the dataset, there are N-1 other points to compare it to, but each pair is counted twice (e.g. AB and BA), so we need to divide the total count by 2 to get the number of unique pairs.

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

N(N-1)/2 = 9(9-1)/2 = 36

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

total pairs: (9 x 8) / 2 = 36

total disagreements: 36 - 9 = 27

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

1. Cluster the data using two different clustering algorithms (e.g. K-means and hierarchical clustering).

2. For each pair of data points, check whether they are in the same cluster for each clustering algorithm. If the two clustering algorithms agree on the cluster assignment, record an agreement. If they disagree, record a disagreement.

3. After checking all pairs of data points, compute the total number of agreements and disagreements for each clustering algorithm.

4. Compute the disagreement distance between the two clustering algorithms as the ratio of disagreements to the total number of pairs of data points.

TIME COMPLEXITY: $O(n^2)$